In [ ]:
#ASHOK KUMAR
#E18CSE029

In [1]:
import pandas as pd
import numpy as np


In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00526/data.zip

--2021-02-02 13:13:32--  https://archive.ics.uci.edu/ml/machine-learning-databases/00526/data.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116104947 (111M) [application/x-httpd-php]
Saving to: ‘data.zip’

data.zip            100%[===================>] 110.73M  82.7MB/s    in 1.3s    

2021-02-02 13:13:34 (82.7 MB/s) - ‘data.zip’ saved [116104947/116104947]



In [3]:
!unzip -q data.zip

In [4]:
from sklearn.model_selection import train_test_split

In [9]:
d = pd.read_csv("BitcoinHeistData.csv")
d.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,100050000.0,princetonCerber
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,100000000.0,princetonLocky
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,200000000.0,princetonCerber
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,71200000.0,princetonCerber
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,200000000.0,princetonLocky


In [10]:
d['label'] = d['label'].apply(lambda x: 0 if x.lower() == 'white' else 1)
d.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,100050000.0,1
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,100000000.0,1
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,200000000.0,1
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,71200000.0,1
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,200000000.0,1


In [11]:
d.insert(1,"sr.no",d.index)
d

,address,sr.no,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,0,2017,11,18,0.008333,1,0,2,1.000500e+08,1
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,1,2016,132,44,0.000244,1,0,1,1.000000e+08,1
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2,2016,246,0,1.000000,1,0,2,2.000000e+08,1
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,3,2016,322,72,0.003906,1,0,2,7.120000e+07,1
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,4,2016,238,144,0.072848,456,0,1,2.000000e+08,1
...,...,...,...,...,...,...,...,...,...,...,...
2916692,12D3trgho1vJ4mGtWBRPyHdMJK96TRYSry,2916692,2018,330,0,0.111111,1,0,1,1.255809e+09,0
2916693,1P7PputTcVkhXBmXBvSD9MJ3UYPsiou1u2,2916693,2018,330,0,1.000000,1,0,1,4.409699e+07,0
2916694,1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw,2916694,2018,330,2,12.000000,6,6,35,2.398267e+09,0
2916695,15iPUJsRNZQZHmZZVwmQ63srsmughCXV4a,2916695,2018,330,0,0.500000,1,0,1,1.780427e+08,0


In [12]:
X = d.iloc[:,1:-1].values
y = d.iloc[:,-1].values

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from joblib import dump, load

In [18]:
def Decision_tree(X,y):
  clf = DecisionTreeClassifier()
  clf = clf.fit(X,y)
  dump(clf, 'clf.joblib')

In [32]:

def knearestn(X,y):
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X, y)
    dump(knn, 'knn.joblib')

In [33]:
import multiprocessing 
import os
from sklearn.metrics import accuracy_score

In [34]:
%%time
p1 = multiprocessing.Process(target=Decision_tree, args = (X_train,y_train)) 
p2 = multiprocessing.Process(target=knearestn, args = (X_train,y_train)) 


p1.start() 
p2.start()  
 
p1.join() 
p2.join()  


CPU times: user 1.62 ms, sys: 25.2 ms, total: 26.8 ms
Wall time: 18.2 s


In [40]:

model = load(f'clf.joblib')
model1= load(f'knn.joblib')

In [46]:
def test(X_test, modelname):
    model = load(f'{modelname}.joblib')
    return model.predict(X_test)

In [52]:
%%time
pool = multiprocessing.Pool(processes=2)

l1 = X_test, "clf"
l2 = X_test, "knn"

result = pool.starmap(test,[l1,l2])



CPU times: user 439 ms, sys: 1.43 s, total: 1.86 s
Wall time: 1min 5s


In [50]:
avg = (result[0] + result[1])//2
print('Testing Accuracy:', accuracy_score(y_test, avg))

Testing Accuracy: 0.9982034478715315
